In [1]:
import pandas as pd
import plotly.graph_objects as go

from const import DATA_DIR

# ============================================================
# 1. LOAD DATA
# ============================================================
file_path = DATA_DIR / "nuclear-weapons" / "number-of-nuclear-weapons-tests" / "number-of-nuclear-weapons-tests.csv"

try:
    df = pd.read_csv(file_path)
    df.columns = [c.strip() for c in df.columns]
    test_col = [c for c in df.columns if "test" in c.lower()][0]
except Exception as e:
    raise FileNotFoundError(f"File not found or error reading {file_path}: {e}")

# ============================================================
# 2. DATA PROCESSING
# ============================================================
df_country = (
    df
    .groupby("Entity")[test_col]
    .sum()
    .sort_values(ascending=True)
    .tail(8)
)

countries = df_country.index.tolist()
values = df_country.values.tolist()

# ============================================================
# 3. COLORS (IDENTICI)
# ============================================================
top_2 = countries[-2:]
colors = [
    "#c0392b" if c in top_2 else "#95a5a6"
    for c in countries
]

# ============================================================
# 4. BAR CHART
# ============================================================
fig = go.Figure()

fig.add_bar(
    x=values,
    y=countries,
    orientation="h",
    marker=dict(color=colors),
    width=0.65,
    hovertemplate="<b>%{y}</b><br>Tests: %{x}<extra></extra>"
)

# ============================================================
# 5. VALUE LABELS (A DESTRA DELLE BARRE)
# ============================================================
max_val = max(values)

for y, v in zip(countries, values):
    fig.add_annotation(
        x=v + max_val * 0.01,
        y=y,
        text=str(int(v)),
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        font=dict(
            size=11,
            color="#555",
            family="Arial",
            weight="bold"
        )
    )

# ============================================================
# 6. CONTEXT ANNOTATION (85%)
# ============================================================
fig.add_annotation(
    x=max_val * 0.7,
    y=countries[2],
    text=(
        "<i>The USA and USSR conducted<br>"
        "<b>85%</b> of all nuclear tests<br>"
        "in history.</i>"
    ),
    showarrow=False,
    align="left",
    font=dict(size=11, color="#c0392b"),
    bgcolor="white",
    bordercolor="#c0392b",
    borderpad=6
)

# ============================================================
# 7. TITLES & LAYOUT
# ============================================================
fig.update_layout(
    title=dict(
        text=(
            "<b>The Cold War Legacy: Nuclear Testing by Country</b><br>"
            "<span style='font-size:13px;color:#7f8c8d;'>"
            "Total confirmed nuclear explosions conducted (Historical Cumulative)"
            "</span>"
        ),
        x=0.02,
        y=0.95,
        xanchor="left"
    ),
    # width=1000,
    # height=600,
    margin=dict(l=160, r=40, t=90, b=40),
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="#2c3e50"
    ),
    xaxis=dict(
        visible=False,
        fixedrange=True
    ),
    yaxis=dict(
        ticks="",
        showline=True,
        linecolor="#333",
        linewidth=1.5,
        fixedrange=True
    )
)


fig.show()


In [2]:
from const import VISUALIZATIONS_DIR

fig.write_html(VISUALIZATIONS_DIR / "cold-war-legacy.html")